## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-14-13-03-46 +0000,nyt,Madagascar’s Embattled President Is Impeached,https://www.nytimes.com/2025/10/14/world/afric...
1,2025-10-14-13-01-29 +0000,nypost,Controversial Nexstar Media-Tegna merger spark...,https://nypost.com/2025/10/14/business/nexstar...
2,2025-10-14-13-00-40 +0000,bbc,"UK will be second-fastest-growing G7 economy, ...",https://www.bbc.com/news/articles/cn092p27xn0o...
3,2025-10-14-13-00-06 +0000,nyt,Global Growth Remains Sluggish as Tariff Threa...,https://www.nytimes.com/2025/10/14/business/im...
4,2025-10-14-13-00-00 +0000,missionlocal,Timely signage for an aging Thunderbird,https://missionlocal.org/2025/10/sf-photos-tim...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2317/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
37,trump,56
88,gaza,24
2,president,17
92,israel,17
280,hostages,16


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
285,2025-10-13-17-58-10 +0000,nypost,Trump boasts Gaza peace process could be presi...,https://nypost.com/2025/10/13/us-news/trump-sa...,138
242,2025-10-13-20-53-37 +0000,wapo,Trump and world leaders meet in Egypt on futur...,https://www.washingtonpost.com/world/2025/10/1...,138
246,2025-10-13-20-45-44 +0000,nypost,Dems largely quiet about Trump’s role in histo...,https://nypost.com/2025/10/13/us-news/top-dems...,135
273,2025-10-13-18-40-17 +0000,nypost,Trump trolls Norway rep over Nobel Peace Prize...,https://nypost.com/2025/10/13/us-news/trump-tr...,132
57,2025-10-14-10-52-43 +0000,nypost,President Trump accuses Time magazine of delib...,https://nypost.com/2025/10/14/us-news/presiden...,126


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
285,138,2025-10-13-17-58-10 +0000,nypost,Trump boasts Gaza peace process could be presi...,https://nypost.com/2025/10/13/us-news/trump-sa...
267,67,2025-10-13-19-08-18 +0000,nypost,Israeli hostages returning to their families l...,https://nypost.com/2025/10/13/world-news/israe...
54,56,2025-10-14-11-00-00 +0000,wsj,Maine’s Democratic Gov. Janet Mills officially...,https://www.wsj.com/politics/elections/susan-c...
156,54,2025-10-14-04-06-54 +0000,nypost,Mamdani’s biggest backer Democratic Socialists...,https://nypost.com/2025/10/14/us-news/zohran-m...
325,53,2025-10-13-14-55-52 +0000,nypost,JPMorgan to invest $10B in areas ‘critical’ to...,https://nypost.com/2025/10/13/business/jpmorga...
170,52,2025-10-14-01-59-18 +0000,cbc,"New York Times, AP, Reuters and Newsmax among ...",https://www.cbc.ca/news/world/new-york-times-a...
166,46,2025-10-14-02-41-46 +0000,nypost,"Ex-student allegedly stabs 3 staff members, pl...",https://nypost.com/2025/10/13/us-news/ex-stude...
6,42,2025-10-14-13-00-00 +0000,wsj,"Tariffs, inflation and other threats such as e...",https://www.wsj.com/economy/trade-tensions-wei...
218,42,2025-10-13-21-59-00 +0000,wsj,What has Trump done as president so far? The W...,https://www.wsj.com/politics/policy/donald-tru...
137,38,2025-10-14-05-51-43 +0000,cbc,"Dozens dead, missing, thousands of homes destr...",https://www.cbc.ca/news/world/mexico-floods-ra...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
